Modelo CNN-LSTM com Otimização de Hiperparâmetros
funcionando com PyTorch e Optuna

In [1]:
import os
import logging
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from datetime import datetime
from copy import deepcopy as dc
from torch.utils.data import DataLoader
from torch.cuda.amp import autocast, GradScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
data_hoje = datetime.now().strftime('%d-%m')
inicio_execucao = pd.Timestamp.now()

os.makedirs(f'../logs/{data_hoje}', exist_ok=True)
os.makedirs(f'../plots/{data_hoje}', exist_ok=True)
os.makedirs(f'../best_models/{data_hoje}', exist_ok=True)

logging.basicConfig(filename=f'../logs/{data_hoje}/bilstm_optuna.log', level=logging.INFO, format='- %(message)s')
logging.info('-' * 50)
logging.info(f'{inicio_execucao} - Iniciando o processo de otimização e treinamento do modelo BiLSTM')

df_original = pd.read_csv('../dados_tratados/combinado/Piratininga/Piratininga_tratado_combinado.csv',
                          usecols=['PM2.5', 'Data e Hora', 'PM10', 'Monóxido de Carbono'], low_memory=False)

df_original['Data e Hora'] = pd.to_datetime(df_original['Data e Hora'], format='%Y-%m-%d %H:%M:%S')
df_original.index = df_original['Data e Hora']
df_original.sort_index(inplace=True)

colunas_selecionadas = ['PM2.5', 'PM10', 'Monóxido de Carbono']
logging.info(f"Colunas selecionadas: {colunas_selecionadas}")

df = df_original[colunas_selecionadas]
df = df.loc['2019-01-01':'2022-01-01']

logging.info(f"Período de análise: 2019-01-01 a 2022-01-01")
df = df.apply(pd.to_numeric, errors='coerce')

In [2]:
def log_imputation(method_name, impute_function, df):
    df_imputed = impute_function(df)
    logging.info(f"Imputação realizada usando: {method_name}")
    return df_imputed

def Mice_imputer(df):
    mice_imputer = IterativeImputer(
        estimator=ExtraTreesRegressor(n_estimators=100, random_state=42),
        max_iter=10,
        random_state=42,
        n_nearest_features=None,
        initial_strategy='mean'
    )
    
    df_imputed = pd.DataFrame(
        mice_imputer.fit_transform(df),
        columns=df.columns,
        index=df.index
    )
    
    return df_imputed

def mean_imputer(df):
    imputer = SimpleImputer(strategy='mean')
    df_imputed = pd.DataFrame(
        imputer.fit_transform(df),
        columns=df.columns,
        index=df.index
    )
    return df_imputed

def median_imputer(df):
    imputer = SimpleImputer(strategy='median')
    df_imputed = pd.DataFrame(
        imputer.fit_transform(df),
        columns=df.columns,
        index=df.index
    )
    return df_imputed

def forward_fill_imputer(df):
    df_imputed = df.fillna(method='ffill')
    return df_imputed

def backward_fill_imputer(df):
    df_imputed = df.fillna(method='bfill')
    return df_imputed

def linear_interpolation_imputer(df):
    df_imputed = df.interpolate(method='linear')
    return df_imputed

def exponential_smoothing_imputer(df):
    df_imputed = df.ewm(alpha=0.5).mean()
    return df_imputed

def knn_imputer(df):
    imputer = KNNImputer(n_neighbors=5)
    df_imputed = pd.DataFrame(
        imputer.fit_transform(df),
        columns=df.columns,
        index=df.index
    )
    return df_imputed

# df_imputed = log_imputation('Mean', mean_imputer, df)
# df_imputed = log_imputation('Median', median_imputer, df)
# df_imputed = log_imputation('Forward Fill', forward_fill_imputer, df)
# df_imputed = log_imputation('Backward Fill', backward_fill_imputer, df)
# df_imputed = log_imputation('Linear Interpolation', linear_interpolation_imputer, df)
# df_imputed = log_imputation('Exponential Smoothing', exponential_smoothing_imputer, df)
# df_imputed = log_imputation('KNN', knn_imputer, df)
df_imputed = log_imputation('MICE', Mice_imputer, df)


C:\dev\scripts\pm25-plots\venv\Lib\site-packages\sklearn\impute\_iterative.py:825: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [3]:
def add_cyclical_features(df):
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['day_of_week'] = df.index.dayofweek

    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
    df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

    df.drop(['hour', 'day', 'month', 'day_of_week'], axis=1, inplace=True)
    return df


df_imputed = add_cyclical_features(df_imputed)

logging.info(f"Adicionadas features cíclicas")

def prepare_dataframe_for_lstm(df, n_steps):
    df = dc(df)
    for col in df.columns:
        for i in range(1, n_steps + 1):
            df[f'{col}(t-{i})'] = df[col].shift(i)
    df.dropna(inplace=True)
    return df


lookback = 8
shifted_df = prepare_dataframe_for_lstm(df_imputed, lookback)

preprocessing_scaler = StandardScaler()
preprocessing_scaler.fit(shifted_df)
shifted_df_as_np = preprocessing_scaler.transform(shifted_df)

X = shifted_df_as_np[:, len(colunas_selecionadas):]
y = shifted_df_as_np[:, 0]

X = dc(np.flip(X, axis=1))

train_split = int(len(X) * 0.7)
val_split = int(len(X) * 0.85)

X_train, X_val, X_test = X[:train_split], X[train_split:val_split], X[val_split:]
y_train, y_val, y_test = y[:train_split], y[train_split:val_split], y[val_split:]

X_train = X_train.reshape((-1, lookback, X_train.shape[1] // lookback))
X_val = X_val.reshape((-1, lookback, X_val.shape[1] // lookback))
X_test = X_test.reshape((-1, lookback, X_test.shape[1] // lookback))
y_train = y_train.reshape((-1, 1))
y_val = y_val.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_val = torch.tensor(X_val).float()
y_val = torch.tensor(y_val).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()


class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


class CNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, kernel_size, out_channels):
        super(CNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_size, out_channels=out_channels, kernel_size=kernel_size, padding=kernel_size // 2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        
        self.lstm = nn.LSTM(out_channels, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape to (batch_size, input_size, sequence_length)
        x = self.cnn(x)
        x = x.permute(0, 2, 1)  # Change shape back to (batch_size, sequence_length, out_channels)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


def objective(trial):
    hidden_size = trial.suggest_int('hidden_size', 16, 128)
    num_layers = trial.suggest_int('num_layers', 1, 5)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [128])
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-2, log=True)
    kernel_size = trial.suggest_categorical('kernel_size', [3, 5])
    out_channels = trial.suggest_int('out_channels', 16, 64)

    model = CNN_LSTM(input_size=X_train.shape[2], hidden_size=hidden_size,
                     num_layers=num_layers, dropout=dropout,
                     kernel_size=kernel_size, out_channels=out_channels).to(device)

    criterion = nn.HuberLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    train_loader = DataLoader(TimeSeriesDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TimeSeriesDataset(X_val, y_val), batch_size=batch_size)

    for epoch in range(100):
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                val_loss += criterion(outputs, batch_y).item()

        val_loss /= len(val_loader)

        intermediate_value = val_loss
        trial.report(intermediate_value, epoch)

        if trial.should_prune():
            raise optuna.TrialPruned()

    return val_loss


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=2)

print('Best trial:')
trial = study.best_trial
print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

best_params = study.best_params
final_model = CNN_LSTM(input_size=X_train.shape[2], hidden_size=best_params['hidden_size'],
                          num_layers=best_params['num_layers'], dropout=best_params['dropout'],
                          kernel_size=best_params['kernel_size'], out_channels=best_params['out_channels']).to(device)


criterion = nn.HuberLoss()
optimizer = optim.AdamW(final_model.parameters(), lr=best_params['learning_rate'],
                        weight_decay=best_params['weight_decay'])

scaler = GradScaler()
num_workers = 0

train_loader = DataLoader(TimeSeriesDataset(X_train, y_train), batch_size=best_params['batch_size'], shuffle=True,
                          num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(TimeSeriesDataset(X_val, y_val), batch_size=best_params['batch_size'], num_workers=num_workers,
                        pin_memory=True)

num_epochs = 500
best_val_loss = float('inf')
patience = 50
no_improve = 0

for epoch in range(num_epochs):
    final_model.train()
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()

        # Usar precisão mista para forward pass
        with autocast():
            outputs = final_model(batch_X)
            loss = criterion(outputs, batch_y)

        # Escalar a perda e fazer o backward pass
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    final_model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            with autocast():
                outputs = final_model(batch_X)
                val_loss += criterion(outputs, batch_y).item()

    val_loss /= len(val_loader)
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improve = 0
        torch.save(final_model.state_dict(), f'../best_models/{data_hoje}/best_model.pth')
    else:
        no_improve += 1

    if no_improve >= patience:
        print(f'Early stopping triggered after {epoch + 1} epochs')
        break

[I 2024-08-29 17:58:47,587] A new study created in memory with name: no-name-aa5748a8-18b3-481e-86de-72b193f69968
C:\dev\scripts\pm25-plots\venv\Lib\site-packages\torch\nn\modules\rnn.py:88: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.45799675946653634 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
[I 2024-08-29 17:59:55,423] Trial 0 finished with value: 0.24736639857292175 and parameters: {'hidden_size': 29, 'num_layers': 1, 'dropout': 0.45799675946653634, 'learning_rate': 0.0070241399842261056, 'batch_size': 128, 'weight_decay': 0.0007313241675339971, 'kernel_size': 3, 'out_channels': 51}. Best is trial 0 with value: 0.24736639857292175.
[I 2024-08-29 18:07:22,334] Trial 1 finished with value: 0.22242229912550218 and parameters: {'hidden_size': 95, 'num_layers': 5, 'dropout': 0.4604863119293005, 'learning_rate': 0.0030573174243804036, 'batch_si

Best trial:
Value:  0.22242229912550218
Params: 
    hidden_size: 95
    num_layers: 5
    dropout: 0.4604863119293005
    learning_rate: 0.0030573174243804036
    batch_size: 128
    weight_decay: 9.506893388559688e-05
    kernel_size: 5
    out_channels: 56


C:\Users\portes\AppData\Local\Temp\ipykernel_10372\2564181523.py:210: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/500, Validation Loss: 0.1711
Epoch 2/500, Validation Loss: 0.1672
Epoch 3/500, Validation Loss: 0.1644
Epoch 4/500, Validation Loss: 0.1721
Epoch 5/500, Validation Loss: 0.1626
Epoch 6/500, Validation Loss: 0.1642
Epoch 7/500, Validation Loss: 0.1666
Epoch 8/500, Validation Loss: 0.1671
Epoch 9/500, Validation Loss: 0.1640
Epoch 10/500, Validation Loss: 0.1680
Epoch 11/500, Validation Loss: 0.1657
Epoch 12/500, Validation Loss: 0.1685
Epoch 13/500, Validation Loss: 0.1736
Epoch 14/500, Validation Loss: 0.1794
Epoch 15/500, Validation Loss: 0.1722
Epoch 16/500, Validation Loss: 0.1706
Epoch 17/500, Validation Loss: 0.1740
Epoch 18/500, Validation Loss: 0.1745
Epoch 19/500, Validation Loss: 0.1775
Epoch 20/500, Validation Loss: 0.1834
Epoch 21/500, Validation Loss: 0.1810
Epoch 22/500, Validation Loss: 0.1787
Epoch 23/500, Validation Loss: 0.1891
Epoch 24/500, Validation Loss: 0.1951
Epoch 25/500, Validation Loss: 0.1897
Epoch 26/500, Validation Loss: 0.1915
Epoch 27/500, Validat

In [4]:
final_model.load_state_dict(torch.load(f'../best_models/{data_hoje}/best_model.pth'))

test_loader = DataLoader(TimeSeriesDataset(X_test, y_test), batch_size=best_params['batch_size'])
final_model.eval()
test_loss = 0
predictions = []
actuals = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = final_model(batch_X)
        test_loss += criterion(outputs, batch_y).item()
        predictions.extend(outputs.cpu().numpy().squeeze())
        actuals.extend(batch_y.cpu().numpy().squeeze())

test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')

# Desnormalização
def inverse_transform_data(data):
    if data.ndim == 1:
        data = data.reshape(-1, 1)
    dummies = np.zeros((data.shape[0], shifted_df_as_np.shape[1]))
    dummies[:, 0] = data.ravel()  # Use ravel() to ensure 1D array
    dummies = preprocessing_scaler.inverse_transform(dummies)
    return dummies[:, 0]

# Desnormalização
predictions = inverse_transform_data(np.array(predictions))
actuals = inverse_transform_data(np.array(actuals))

# Rest of the code remains the same
mse = mean_squared_error(actuals, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actuals, predictions)
r2 = r2_score(actuals, predictions)
mape = np.mean(np.abs((actuals - predictions) / actuals)) * 100

print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')
print(f'R-squared: {r2:.4f}')
print(f'Mean Absolute Percentage Error: {mape:.4f}')

logging.info(f'Mean Squared Error: {mse:.4f}')
logging.info(f'Root Mean Squared Error: {rmse:.4f}')
logging.info(f'Mean Absolute Error: {mae:.4f}')
logging.info(f'R-squared: {r2:.4f}')
logging.info(f'Mean Absolute Percentage Error: {mape:.4f}')


# Plotar resultados
plt.figure(figsize=(12, 6))
plt.plot(actuals, label='Actual')
plt.plot(predictions, label='Predicted')
plt.title('Actual vs Predicted PM2.5 Values')
plt.xlabel('Time')
plt.ylabel('PM2.5')
plt.legend()
plt.savefig(f'../plots/{data_hoje}/actual_vs_predicted.png')
plt.close()

# Plotar scatter plot
plt.figure(figsize=(10, 10))
plt.scatter(actuals, predictions, alpha=0.5)
plt.plot([min(actuals), max(actuals)], [min(actuals), max(actuals)], 'r--', lw=2)
plt.xlabel('Actual PM2.5')
plt.ylabel('Predicted PM2.5')
plt.title('Actual vs Predicted PM2.5 Scatter Plot')
plt.savefig(f'../plots/{data_hoje}/scatter_plot.png')
plt.close()

# Salvar resultados
results = pd.DataFrame({'Actual': actuals, 'Predicted': predictions})
results.to_csv(f'../results/{data_hoje}/predictions.csv', index=False)

# Salvar métricas
with open(f'../results/{data_hoje}/metrics.txt', 'w') as f:
    f.write(f'Mean Squared Error: {mse:.4f}\n')
    f.write(f'Root Mean Squared Error: {rmse:.4f}\n')
    f.write(f'Mean Absolute Error: {mae:.4f}\n')
    f.write(f'R-squared: {r2:.4f}\n')

print("Análise concluída. Resultados salvos nos diretórios correspondentes.")


C:\Users\portes\AppData\Local\Temp\ipykernel_10372\1532132186.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  final_model.load_state_dict(torch.load(f'../best_models/{da

Test Loss: 0.1522
Mean Squared Error: 27.7366
Root Mean Squared Error: 5.2666
Mean Absolute Error: 3.1793
R-squared: 0.6532
Mean Absolute Percentage Error: 45.2617


OSError: Cannot save file into a non-existent directory: '..\results\29-08'